In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

## Read Bronze table

In [0]:
df = spark.table("workspace.bronze.erp_px_cat_g1v2")

## Silver Transformations

## Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
df.display()

## NO NULL

In [0]:
null_counts = df.select([F.count(F.when(col(c).isNull(), c)).alias(c) for c in df.columns])
display(null_counts)

## Renaming Columns

In [0]:
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

## Normalize Maintenance Flag to Boolean

In [0]:
df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
     .when(F.upper(col("maintenance")) == "NO", F.lit(False))
     .otherwise(None)
)


In [0]:
df.display()

In [0]:
df.limit(10).display()

## Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_product_category")

In [0]:
%sql
SELECT * FROM workspace.silver.erp_product_category LIMIT 10